In [ ]:
# Κελί 1: Εγκατάσταση βιβλιοθηκών
!pip install scikit-image opencv-python tensorflow bm3d numpy matplotlib
# Το BM3D απαιτεί συχνά μια ξεχωριστή εγκατάσταση

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.0/862.0 kB 9.5 MB/s eta 0:00:00


In [ ]:
# Κελί 2: Εισαγωγές
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from skimage.util import random_noise, img_as_float32, img_as_ubyte
from skimage.restoration import denoise_nl_means, estimate_sigma
import bm3d
from google.colab import drive
from tensorflow.keras import layers

# Έλεγχος έκδοσης TensorFlow
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.19.0


In [ ]:
# Κελί 3: Φόρτωση Δεδομένων

drive.mount('/content/drive')

# ΟΡΙΣΜΟΣ ΣΩΣΤΩΝ ΔΙΑΔΡΟΜΩΝ
BASE_DIR = "/content/drive/MyDrive/Datasets"

# ΔΙΟΡΘΩΣΗ CBSD68: Δείχνει στον υποφάκελο 'original_png'
CBSD68_DIR = os.path.join(BASE_DIR, "CBSD68", "original_png")

# ΤΕΛΙΚΗ ΔΙΟΡΘΩΣΗ KODAK:
# kodak_24 (πρώτος φάκελος) -> kodak24 (υποφάκελος, όλα πεζά)
KODAK_DIR  = os.path.join(BASE_DIR, "kodak_24", "kodak24")

def load_dataset(folder, target_size=(256, 256)):
    """Φορτώνει εικόνες, τις μετατρέπει σε float32 και τις αλλάζει μέγεθος."""
    images = []
    if not os.path.isdir(folder):
        print(f"❌ ΣΦΑΛΜΑ: Ο φάκελος δεν βρέθηκε: {folder}")
        return images

    for filename in sorted(os.listdir(folder)):
        # Ελέγχουμε μόνο για PNG/JPG
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)

            if img is None:
                continue

            # Μετατροπή από BGR σε RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Αλλαγή μεγέθους
            img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
            images.append(img_as_float32(img))

    if images:
        return np.array(images)
    else:
        print(f"⚠️ ΠΡΟΣΟΧΗ: Ο φάκελος {folder} βρέθηκε, αλλά δεν περιέχει εικόνες.")
        return images

# Φόρτωση των δεδομένων
cbsd_images = load_dataset(CBSD68_DIR)
kodak_images = load_dataset(KODAK_DIR, target_size=(512, 768))

# Έλεγχος και εκτύπωση αποτελεσμάτων
print("Loaded CBSD68 images (for training):", cbsd_images.shape if isinstance(cbsd_images, np.ndarray) else "0")
print("Loaded Kodak24 images (for testing/comparison):", kodak_images.shape if isinstance(kodak_images, np.ndarray) else "0")

# Ορισμός μεταβλητών για το training και testing
if isinstance(cbsd_images, np.ndarray) and cbsd_images.size > 0:
    TRAIN_IMAGES = cbsd_images
else:
    print("🛑 Αδύνατη η εκπαίδευση: Δεν φορτώθηκαν εικόνες CBSD68.")
    TRAIN_IMAGES = np.array([])

if isinstance(kodak_images, np.ndarray) and kodak_images.size > 0:
    # Επιλέγουμε την πρώτη εικόνα του Kodak για επίδειξη
    TEST_IMAGE = kodak_images[0]
else:
    print("🛑 Δεν φορτώθηκαν εικόνες δοκιμής Kodak. Δημιουργείται μαύρη εικόνα για fallback.")
    TEST_IMAGE = np.zeros((512, 768, 3), dtype=np.float32)

Mounted at /content/drive
Loaded CBSD68 images (for training): (68, 256, 256, 3)
Loaded Kodak24 images (for testing/comparison): (24, 768, 512, 3)


In [ ]:
def add_gaussian(img, sigma=25/255.0):
    return np.clip(img + np.random.normal(0, sigma, img.shape), 0, 1)

def add_salt_pepper(img, amount=0.05):
    return random_noise(img, mode='s&p', amount=amount)

def calculate_psnr(original, noisy):
    """Υπολογίζει το Peak Signal-to-Noise Ratio (PSNR)."""
    # Το MSE υπολογίζεται για τιμές [0, 1]
    mse = np.mean((original - noisy) ** 2)
    if mse == 0:
        return 100 # Εικόνες ίδιες
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

In [ ]:
def apply_nlm(img):
    sigma_est = np.mean(estimate_sigma(img, multichannel=True))
    patch_kw = dict(patch_size=5, patch_distance=6, multichannel=True)
    return denoise_nl_means(img, h=1.15 * sigma_est, fast_mode=True, **patch_kw)

def apply_bm3d(img, sigma=25/255.0):
    return bm3d(img, sigma_psd=sigma)

In [ ]:
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
    return x

def build_unet():
    inputs = layers.Input(shape=(None, None, 3))

    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D()(c1)

    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D()(c2)

    c3 = conv_block(p2, 256)

    u1 = layers.UpSampling2D()(c3)
    u1 = layers.Concatenate()([u1, c2])
    c4 = conv_block(u1, 128)

    u2 = layers.UpSampling2D()(c4)
    u2 = layers.Concatenate()([u2, c1])
    c5 = conv_block(u2, 64)

    outputs = layers.Conv2D(3, 1, padding="same", activation="sigmoid")(c5)

    return Model(inputs, outputs)

unet = build_unet()
unet.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, None,      │      1,792 │ input_layer[0][0] │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, None,      │     36,928 │ conv2d[0][0]      │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, None,      │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, None,      │     73,856 │ max_pooling2d[0]… │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, None,      │    147,584 │ conv2d_2[0][0]    │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, None,      │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, None,      │    295,168 │ max_pooling2d_1[… │
│                     │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, None,      │    590,080 │ conv2d_4[0][0]    │
│                     │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, None,      │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, None,      │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ None, 384)        │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, None,      │    442,496 │ concatenate[0][0] │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, None,      │    147,584 │ conv2d_6[0][0]    │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, None,      │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, None,      │          0 │ up_sampling2d_1[… │
│ (Concatenate)       │ None, 192)        │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, None,      │    110,656 │ concatenate_1[0]… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, None,      │     36,928 │ conv2d_8[0][0]  

 Total params: 1,883,267 (7.18 MB)

 Trainable params: 1,883,267 (7.18 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_clean = np.array(cbsd_images)
train_noisy = np.array([add_gaussian(img, sigma=25/255.0) for img in train_clean])

print(train_clean.shape, train_noisy.shape)

(68, 256, 256, 3) (68, 256, 256, 3)


In [ ]:
unet.compile(optimizer='adam', loss='mse')

history = unet.fit(
    train_noisy, train_clean,
    batch_size=4,
    epochs=15,
    validation_split=0.1
)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 303s 19s/step - loss: 0.0557 - val_loss: 0.0419
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 294s 18s/step - loss: 0.0335 - val_loss: 0.0179
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 333s 19s/step - loss: 0.0168 - val_loss: 0.0240
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 311s 18s/step - loss: 0.0152 - val_loss: 0.0083
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 305s 19s/step - loss: 0.0074 - val_loss: 0.0064
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 293s 18s/step - loss: 0.0059 - val_loss: 0.0054
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 299s 19s/step - loss: 0.0047 - val_loss: 0.0037
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 316s 18s/step - loss: 0.0041 - val_loss: 0.0035
Epoch 9/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 332s 19s/step - loss: 0.0037 - val_loss: 0.0028
Epoch 10/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 292s 18s/step - loss: 0.0032 - val_loss: 0.0026
Epoch 11/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 334s 19s/step - loss: 0.0031 - val_loss: 0.0026
Epoch 12/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 310s 18s/step